In [117]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ddm_model import DDM, Simulation

# Base Model (Pure DDM)

In [135]:
ddm_one = DDM(drift_rate=0.017348, noise_mag=0.1, threshold=15,initial_condition=0.03) #drift rate max is around 0.07.

In [136]:
simulation_one = Simulation(ddm_one, 1, 1000,15)

In [137]:
simulation_one.plot_trajectories()

Max reaction time: 999.0 ms
Simulated timesteps: 1000


In [138]:
simulation_one.reaction_times.mean()

np.float64(851.6)

In [139]:
simulation_one.error_rate

0.13333333333333333

In [140]:
simulation_one.simulated_results

[('None', 'A'),
 ('B', 'B'),
 ('None', 'A'),
 ('A', 'A'),
 ('B', 'B'),
 ('A', 'A'),
 ('A', 'A'),
 ('A', 'A'),
 ('B', 'B'),
 ('A', 'A'),
 ('A', 'A'),
 ('B', 'B'),
 ('A', 'A'),
 ('A', 'A'),
 ('A', 'A')]

Trajectory simulation data:

In [141]:
pd.DataFrame(simulation_one.simulated_trajectories).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.260715,-0.115359,-0.058134,-0.048951,0.005499,-0.074371,-0.130533,-0.062974,-0.052545,-0.082152,0.163195,0.031161,0.006068,0.071263,0.007091
1,0.372584,-0.133392,-0.159681,-0.081871,-0.090948,-0.083503,-0.078256,0.099317,-0.044935,0.010143,0.261743,-0.010138,0.005663,-0.076007,-0.073939
2,0.411495,-0.198246,-0.269952,-0.147232,-0.193303,0.050374,-0.125973,0.088821,-0.176794,0.038243,0.261286,-0.036960,0.014299,-0.082279,0.080426
3,0.237725,-0.236400,-0.127744,-0.273330,-0.198641,0.162339,-0.076265,0.132226,-0.217424,0.118756,0.294203,0.151305,0.073785,-0.037543,0.146283
4,0.154967,-0.144669,-0.107809,-0.321166,-0.214673,0.200458,-0.000610,0.190512,-0.052404,0.196528,0.194800,0.119495,-0.052881,-0.022662,0.297741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10.556139,NaN,11.760625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,10.492894,NaN,11.724776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,10.590491,NaN,11.762136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,10.745857,NaN,11.880718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
